In [1]:
%load_ext twisted_ipython
%autoawait twisted

In [2]:
import attr
from twisted.internet import reactor
from txdbus import client

from pyxsession.dbus import dbus_attr, Str
from pyxsession.dbus import Service


@attr.s
class Payload:
    message = dbus_attr(default='', type=Str())


In [3]:
test_service = Service('org.jfhbrook.testing123')

test_obj = test_service.obj('/TestObj')
test_obj.property('test_prop', Str(), 'some default', writeable=True)
test_obj.signal('message', Payload)

@test_obj.method([Payload], Str())
def echo(payload):
    return f'You said: {payload.message}'

In [4]:
conn = await client.connect(reactor)

In [5]:
server = await test_service.server(conn)

server

Server(connection=<txdbus.client.DBusClientConnection object at 0x7efcac27ccc0>, service=<pyxsession.dbus.service.Service object at 0x7efcac27cef0>, bus_name=1, dbus_obj_cls=<class 'pyxsession.dbus.server.TestObj'>, dbus_obj=<pyxsession.dbus.server.TestObj object at 0x7efcac1b6e48>)

In [6]:
client = await test_service.client(conn)

client

Client(service=<pyxsession.dbus.service.Service object at 0x7efcac27cef0>, remote_objs={'/TestObj': <txdbus.objects.RemoteDBusObject object at 0x7efcac1b6860>})

In [7]:
client.TestObj.on('message', print)

server.TestObj.emit('message', Payload('hello there'))

Payload(message='hello there')
